# Guided Filter

Using implementation by PfChai (https://github.com/pfchai/GuidedFilter) using original paper implementation and code in MATLAB by Kaiming He (http://kaiminghe.com/eccv10/, http://kaiminghe.com/publications/eccv10guidedfilter.pdf). The Python implementation of the code has been reviewed by author for correspondance against original MATLAB code and paper mathematical background.

In [ ]:
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
from guidedfilter import guidedfilter
import glob
import os

In [ ]:
def guided_filter(input_file, output_file):
    print('Begin filtering file ' + input_file)
    
    # inputing the file
    sar_image=gdal.Open(input_file)
    sar_band=sar_image.GetRasterBand(1)
    sar_band.GetMetadata()
    img_array=sar_band.ReadAsArray()
    [cols, rows]=img_array.shape
    
    filtered_image=guidedfilter(img_array, img_array,  r=4, eps=0.0002)
    
    # writing output file
    driver = gdal.GetDriverByName("GTiff")
    output = driver.Create(output_file, rows, cols, 1, gdal.GDT_Float32)
    proj = output.SetGeoTransform(sar_image.GetGeoTransform())  
    output.SetProjection(sar_image.GetProjection())
    output.SetGCPs(sar_image.GetGCPs(), "4326")
    geoband = output.GetRasterBand(1)
    geoband.WriteArray(filtered_image)
    output.GetRasterBand(1).SetNoDataValue(-9999)
    output.FlushCache()
    output = None
    band=None
    print('Created file ' + output_file)
    
    print('End filtering file ' + input_file)  

In [ ]:
for input_file in glob.glob('/home/cristina/Seeps/Workflow/Filters/ROI/*/NRCS/*tif'):
    output_file = input_file.split(os.sep)
    output_file[-1] = output_file[-1][:-4] + '_GF_4_0002.tif'
    output_file = output_file[:-2] + ['GuidedFilter'] + output_file[-2:]
    output_file = os.sep.join(output_file)
    guided_filter(input_file, output_file)
print('All files have been finished!')